数值预测的含义

在上一章节，我们接触到的决策树，比较适合对数据的分类进行预测，以及我们之前学过的分类器也是如此。但是当我们对数值型结果进行预测的时候应该怎么办呢？
具体什么叫做对数值型结果进行预测首先需要明确一下。比如：我们要在拍卖行竞价购买一个笔记本电脑，这台笔记本电脑有一些参数：处理器的速度，RAM的容量，硬盘的大小，屏幕的分辨率以及其他因素。显然，我们最终对其的定价必然要考虑这些参数，而这些参数的重要性各不相同，比如硬盘大小与屏幕大小相比，可能大家都觉得屏幕大小更为重要。那么各个因素都影响着我们最终对该款笔记本的最终的定价，这个定价就是我们所说的对数值型结果进行预测。我们可以使用第五章研究过的优化技术，求出最佳的权重。书中提出：

    贝叶斯分类器
    决策树
    支持向量机

都不是应对这种情况的最佳算法。本章将会研究如何应对这样的情况。


构建数据集

理解我们的要做什么之后，首先要做的就是数据集的问题，这次很数据集的来源很不一样，是我们自己构建的。
在我们构建数据集的时候，不得不注意这个数据集必须具有某些特征（也即是处理器速度、硬盘大小），而且这些特征最好比较复杂，使得价格比较难以预测。比如如果对电视机进行价格的预测，很显然屏幕越大，价格越高，那么这个预测实在是太简单了。
本书提出，构建的是一个葡萄酒价格的数据集。首先需要明确，酒的价格由两点决定，

    等级
    储藏的年代

此外，葡萄酒还有“峰值年”这样一种说法，简单说来不同等级的酒，都会到了一个年份，到了该年份酒的价格是最高的，而接近这个年份的时候价格会增加，过了这个年份价格会逐渐降低。
请看代码：

In [17]:
from random import random,randint
import math
#rating代表酒的等级，age代表酒的年代
#如果rating是代表酒的等级，同一等级的酒的峰值年是一样的。     
#所以每一个峰值年是针对同一类等级的酒而言
def wineprice(rating,age):    
    peak_age = rating - 50 
    
    #根据等级来计算价格
    price = rating/2
    if age>peak_age:
        #经过“峰值年”之后，之后的5年，酒的品质会变差，价格降低
        price = price * (5-(age-peak_age))
        
    else:
        #价格在接近“峰值年”时，会增加到原值的五倍
        price = price*(5*((age+1)/peak_age))
    if price < 0: price=0
    return price

def wineset1():
    rows = []
    for i in range(300):
        #随机产生年代和等级
        rating = random()*50+50
        age = random()*50
        
        #得到一个参考价格 
        price = wineprice(rating,age)
        
        #增加“噪声”，也就是让酒的价格随机波动一下
        price*=(random()*0.4+0.8) #这个写法很高端
        
        #加入到数据集中
        rows.append({'input':(rating,age),
                        'result':price})
        return rows  


#用欧几里得来计算两瓶酒的相似度
def euclidean(v1,v2):
    d = 0.0
    for i in range(len(v1)):
        d += (v1[i] - v2[i])**2
    return math.sqrt(d)
    
    

执行代码，同一年等级的酒，不同的age:

In [4]:
print ('(55,8):',wineprice(55,8))
print ('(55,9):',wineprice(55,9))

(55,8): 55.0
(55,9): 27.5


我们可以看到，对于等级55的酒来说，峰值年为5，所以对于两个过了峰值年的酒来说，第9年显然比第8年更便宜。
接着，我们来用代码产生葡萄酒价格的数据集，代码会随机产生200个普通酒的价格和年份，并且计算出其价格，然后随机加减20%，可以理解为是税收和价格的变动。
代码如下：

In [26]:
def wineset1():
    rows = []
    for i in range(300):
        #随机产生年代和等级
        rating = random()*50+50
        age = random()*50
        
        #得到一个参考价格 
        price = wineprice(rating,age)
        
        #增加“噪声”，也就是让酒的价格随机波动一下
        price*=(random()*0.4+0.8) #这个写法很高端
        
        #加入到数据集中
        rows.append({'input':(rating,age),
                        'result':price})
    return rows  

In [27]:
wineprice(95,3)

21.111111111111114

In [28]:
wineprice(95,8)

47.5

In [29]:
wineprice(99,1)

10.102040816326529

In [30]:
data = wineset1()

In [31]:
data[0]

{'input': (61.34411098690677, 6.246561129309258), 'result': 89.32909526226072}

In [32]:
data[1]

{'input': (58.14307938173174, 21.401432710781), 'result': 0.0}

有了数据集之后，我们就研究如何对一瓶新的普通酒进行价格预测。虽然，我们在构建数据集的时候使用了一个函数来计算出价格，我们心里也知道这个价格也许是虚构的，但是，现在请将数据集里产生的价格认为是真实的，这样，我们才能对一瓶新的普通酒进行价格预测，而不是直接用之前的函数wineprice（）算出价格。

k-最近邻算法

这个算法思想来自于一个简单的事实：我们会找到和新普通酒相似的酒，然后看看这个相似酒的价格，再推算我们新酒的价格。所以，该算法会寻找一组与新普通酒相似的酒，求出这些价格的均值，做出价格预测。这样方法就是k-nearest nerghbors，kNNE：k-最近邻算法。
上面一组，就代表了几个与新酒相似度的酒，这也是k的含义。那么到底选几个呢？这是值得探究的问题，显然选少了或者选多了都不是不行的。我们在实际运用时可以多试试不同的k值，也许会得到更为准确的结果。
如何确定两瓶酒相似呢？这里我们使用了应该是比较简单的算法：欧几里得算法

代码如下：

In [16]:
#用欧几里得来计算两瓶酒的相似度
def euclidean(v1,v2):
    d = 0.0
    for i in range(len(v1)):
        d += (v1[i] - v2[i])**2
    return math.sqrt(d)

In [24]:
rows=wineset1()
print (rows[0])
print (rows[1])
print (euclidean(rows[0]['input'],rows[1]['input']))

{'input': (69.23773040631832, 15.127324831115342), 'result': 145.92963650364862}
{'input': (87.1562786489632, 22.91779374092223), 'result': 167.53160082354157}
19.53882742025886


有了相似度计算公式之后，我们很容易就能够计算出两瓶酒的相似度了，下面的代码是用于计算需要预测的新酒和数据集中的每一个的酒的距离（也就是相似度），算出来之后，我们才能排序，抽取出其中k个最相似度。注意，该函数的计算量比较大。
代码如下：

In [36]:
#得到需要预测的新酒与数据集中所有酒的相似度
def getdistances(data,vec1):
    distancelist = []
    for i in range(len(data)):
        vec2 = data[i]['input']
        distancelist.append((euclidean(vec1,vec2),i))
    distancelist.sort()
    return distancelist

拿到了新酒与所有酒的相似度，我们取出最相似的k个，算出这个k个酒的平均值，我们就得到了对新酒预测的价格：

In [37]:
def knnestimate(data,vec1,k=5):
    #得到排序过后的相似度排序
    dist = getdistances(data,vec1)
    avg = 0.0
    #对前k项结果求平均值
    for i in range(k):
        idx = dist[i][1] #这里地方之所以是1的原因是取出在data列表里的序号
        avg += data[idx]['result']
    avg = avg/k
    return avg

In [38]:
knnestimate(data,(95.0,3.0))

16.98100614552323

上面的结果是使用了默认的k为5，那么不同的k，产生的结果肯定也是不一样的。

最相近的酒应该占有最大的比重

我们发现一个问题，就是当我们的k为5的时候，相似的五瓶酒在求出最后新酒的平均值的时候占的比重是一样的，我们需要，越相近的酒占的比重应该更大，这样结果才准确。

所以，我们要将得到相似度转化为权重。书中介绍了集中方式来完成这个功能：


inverse function


书中对这个词的翻译应该有错，书上想讲的是y=1/x的这样的函数，却翻译为了反函数。
使用inverse function就可以完成将距离转换为权重这个过程。因为用欧几里得算出来的是两个点之间的距离，如果距离越大，那么其倒数就越小，如果距离越近，那么其倒数越大。这个方法有一个特点，就是如果非常近的话，那么占的权重非常之大，以至于会忽略掉相距稍稍有一点远的邻居，而且相距有点点远，但是它所在占的比重会下降的非常快。这到底是好事还是坏事，要看具体的项目有什么要求。

In [40]:
#使用倒数来将距离转为权重
#const的存在是为了防止两点非常近，而导致了其距离非常近，倒数特别大，大到其他数都不起作用
def inverseweight(dist,num=1.0,const=0.1):
    return num/(dist + const)

减法函数

我们也可以使用减法函数来完成将距离转化为权重。思路也非常简单，用一个固定的数去减去距离，如果距离越大，那么返回数就越小，如果距离大过某个程度，那么就返回0了。这个办法的坏处就是，当大多数邻居的距离都比较大的时候，如果都返回了0，就会导致数据不足，就没办法预测了。
代码如下：

In [41]:
#用减法函数将距离转化为权重
def subtractweight(dist,const=1.0):
    if dist > const:
        return 0
    else:
        return const - dist

高斯函数

高斯函数也是将距离转化为权重的方法。思路涉及原理，也就是正态分布“钟形曲线”，这就不讲解了，说白了也是带入公式而已。但是高斯函数克服了上述缺点，比如权重是始终不会跌至0的。看下图，高斯函数的图形就会明白了，x轴为距离，y轴为比重：
代码如下：

In [42]:
def gaussian(dist,sigma=10.0):
    return math.e**(-dist**2/(2*sigma**2))

In [43]:
print (inverseweight(0.1))
print (subtractweight(0.1))
print (gaussian(0.1))

5.0
0.9
0.9999500012499791


In [44]:
print (inverseweight(1))
print (subtractweight(1))
print (gaussian(1))

0.9090909090909091
0.0
0.9950124791926823


测试三个函数的执行代码看看三者之间的差异

加权kNN

实际上没什么复杂了，和之前推荐的内容类似。刚刚我们的kNN求的是平均，这次我们就是要求加权平均。也就是通过每一项的值乘以权重，然后将结果累加而得到的。总和再除以权重之和即可。
代码：

In [45]:
def weightedknn(data,vec1,k=5,weightf=gaussian):
    #得到距离值
    dlist=getdistances(data,vec1)
    avg = 0.0
    totalweight = 0.0
    
    #得到加权平均值
    for i in range(k):
        dist = dlist[i][0]
        idx = dlist[i][1]
        weight = weightf(dist)
        avg += weight*data[idx]['result']
        totalweight += weight
    avg = avg/totalweight
    return avg

In [46]:
weightedknn(data,(99.0,5.0))

25.595451463064162

In [47]:
knnestimate(data,(99.0,5.0))

25.494485701165587

交叉验证

英文为：Cross-validation。这项技术将数据集拆分为训练集和测试集（都带有正确的答案），然后我们用训练集去训练模型，该训练集应该是带有正确答案的，然后我们再将测试集的输入传入算法，得到输出。将输出与正确的答案进行比对，看输出与正确的答案差距有多少。这里是我个人的理解，书中的描述我认为是有点错误哈。拆分一般都9比1的方式来拆分，显然训练集需要多一些。
这样有什么好处？就是验证我们的算法是否能够准确预测，而且我们可以对比使用不同的参数产生的结果，比如k的数量，比如到底是使用减法函数还是高斯函数来做距离转化为权重。
但是，我们这里并不是拿训练集去算法模型，而且用训练集去产生测试集中的一个预测的结果，比如knnestimate(data,vec1,k=5) 函数，其中的data传入的就是我们的训练集，vec1就是测试集中的一个。这是这一次算法的需要，但是我认为本质上还是没变的。
首先是把一个数据集划分为2个数据集，一个训练集和一个测试集，需要95%的训练集，5%的测试集。
代码如下：

In [50]:
#将数据拆分为训练集和测试集
def dividedata(data,test=0.05):
    trainset = []
    testset = []
    for row in data:
        if random() < test:
            testset.append(row)
        else:
            trainset.append(row)
    return trainset,testset

我们去测试算法，会得到产生对算法预测的误差。注意，本函数中，我们统计了是差值的平方，而不是单纯的差值。两者各有好处。
如果我们想突显偶尔出现一次很大的差距，使用差值的平方
如果我们关心每次与正确值的差距，而偶尔有一次很大的差距也无所谓，那么使用差值绝对值相加
测试算法的代码：

In [52]:
#测试算法的误差
#而是直接将训练集传入knnestimate函数，作为产生一个预测结果的基础，然后算出预测结果和真实结果之间的差距
def testalgorithm(algf,trainset,testset):
    error = 0.0
    for row in testset:#这里只是拿testset来做个循环
        guess = algf(trainset,row['input'])
        error += (row['result']-guess)**2 #对数字求平方这样会突显其差距。
    return error/len(testset)

交叉测试的控制代码，从代码中，我们看出来了，并不是只做了一次拆分数据集和测试数据集代码的工作，我们是重复了100次，然后再取平均值。
代码如下：

In [53]:
def crossvalidate(algf,data,trials=100,test=0.05): 
    error=0.0 
    for i in range(trials):
        trainset,testset=dividedata(data,test)
        error+=testalgorithm(algf,trainset,testset)
    return error/trials

In [54]:
crossvalidate(knnestimate,data)

430.2308244143351

In [55]:
def knn3(d,v):
    return knnestimate(d,v,k=3)

In [56]:
crossvalidate(knn3,data)

481.1702480611702

In [58]:
def knn5(d,v):
    return knnestimate(d,v,k=5)

In [59]:
crossvalidate(knn5,data)

488.8274665467399

>>> ================================ RESTART ================================ >>>  k=3时算法的误差： 534.299703506 k=3时算法的误差： 422.359768538 k=3时算法的误差： 460.892823922 k=3时算法的误差： 561.394791352 k=3时算法的误差： 438.566549999 >>> ================================ RESTART ================================ >>>  k=5时算法的误差： 356.420358448 k=5时算法的误差： 371.83561953 k=5时算法的误差： 299.178929108 k=5时算法的误差： 391.072240086 k=5时算法的误差： 352.400721703 >>> 

我们发现k=5时，误差要低很多，所以觉得k=5不错。